In [12]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Função de normalização dos dados
def normalize_data(data):
    data_min = np.min(data, axis=0)
    data_max = np.max(data, axis=0)
    data = (data - data_min) / (data_max - data_min)
    return data

In [13]:
# Definição da arquitetura do autoencoder
def build_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='softmax')(encoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)

    # Define o encoder e decoder separadamente
    encoder = Model(inputs=input_layer, outputs=encoded)
    encoded_input = Input(shape=(encoding_dim,))
    decoder_layer = autoencoder.layers[-1]
    decoder = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))

    autoencoder.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

    # Adiciona propriedades ao autoencoder
    autoencoder.encoder = encoder
    autoencoder.decoder = decoder

    return autoencoder


In [14]:
# Função de treinamento com Early Stopping
def train_autoencoder_with_early_stopping(autoencoder, data, labels, val_data, val_labels, epochs=100, patience=10):
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, restore_best_weights=True)
    history = autoencoder.fit(data, labels, epochs=epochs, batch_size=32, validation_data=(val_data, val_labels), callbacks=[early_stopping])
    return history

In [16]:
# Função de plotagem da evolução do treinamento
def plot_training_history(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()


In [17]:
# Função de teste para calcular a SER
def test_autoencoder_with_errors(autoencoder, data, labels, EbNodB_range, R):
    ber = np.zeros(len(EbNodB_range))
    num_errors = []

    for n in range(len(EbNodB_range)):
        EbNo = 10.0 ** (EbNodB_range[n] / 10.0)
        noise_std = np.sqrt(1 / (2 * R * EbNo))
        noise = np.random.normal(0, noise_std, data.shape)

        encoded_signal = autoencoder.encoder.predict(data)
        channel_signal = encoded_signal + noise
        decoded_signal = autoencoder.decoder.predict(channel_signal)

        pred_output = np.argmax(decoded_signal, axis=1)
        errors = np.sum(pred_output != labels)
        ber[n] = errors / len(labels)
        num_errors.append(errors)

        print(f'SNR: {EbNodB_range[n]} BLER: {ber[n]} no_errors: {errors}')

    return ber, num_errors

In [18]:
# Função para visualizar o resultado do encoder
def plot_encoded_space(encoder, data, labels):
    encoded_data = encoder.predict(data)
    plt.figure(figsize=(10, 6))
    plt.scatter(encoded_data[:, 0], encoded_data[:, 1], c=labels, cmap='viridis')
    plt.colorbar()
    plt.xlabel('Encoded Dimension 1')
    plt.ylabel('Encoded Dimension 2')
    plt.title('Encoded Space')
    plt.grid(True)
    plt.show()

In [19]:
# Função de definição dos parâmetros de treino
def prepare_data(n, k):
    data_size = 1000
    data = np.random.randint(0, 2, size=(data_size, n))
    labels = np.packbits(data, axis=-1)
    labels = np.squeeze(labels)

    val_data = np.random.randint(0, 2, size=(200, n))
    val_labels = np.packbits(val_data, axis=-1)
    val_labels = np.squeeze(val_labels)

    data = normalize_data(data)
    val_data = normalize_data(val_data)

    return data, labels, val_data, val_labels

In [20]:
# Função de plotagem do número de erros por SNR
def plot_errors_by_snr(num_errors, EbNodB_range):
    plt.figure(figsize=(10, 6))
    plt.plot(EbNodB_range, num_errors, marker='o', label='Number of Errors')
    plt.xlabel('SNR (dB)')
    plt.ylabel('Number of Errors')
    plt.title('Number of Errors vs SNR')
    plt.legend()
    plt.grid(True)
    plt.show()

In [21]:
# Parâmetros gerais
EbNodB_range = np.linspace(0, 10, 11)
R = 1

In [22]:
# Exercício a: n=1, k=2, modulação PSK
data_a, labels_a, val_data_a, val_labels_a = prepare_data(1, 2)
autoencoder_a = build_autoencoder(1, 2)
history_a = train_autoencoder_with_early_stopping(autoencoder_a, data_a, labels_a, val_data_a, val_labels_a)
plot_training_history(history_a)
ber_a, num_errors_a = test_autoencoder_with_errors(autoencoder_a, data_a, labels_a, EbNodB_range, R)
plot_encoded_space(autoencoder_a.encoder, data_a, labels_a)
plot_errors_by_snr(num_errors_a, EbNodB_range)

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-